# Download Sentinel-2 Data from EarthDaily Analytics' Earth Platform

This notebook provides a guide for downloading [Sentinel-2](https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-2) data from the [EarthDaily Analytics Earth Platform](https://earthplatform.eds.earthdaily.com/) and displaying some of the data downloaded from this platform.

Ideally, this is a template for downloading Sentinel-2 data for an area **you** are interested in learning more about and inspiring **you** to perform your own analysis based on your own personal whimsy.

We start by introducing techniques for querying the Platform by using a GeoJSON Polygon created for the Lower Mainland; recommend how you can go about creating your own; and then proceed with an example for querying and displaying data from this store.

### What you Need to Run this Notebook:

(1) A file located in the same directory as this notebook called `.env` containing the following format and values:

```bash
CLIENT_ID="very_real_client_id_for_earthdaily_analytics_platform"
CLIENT_SECRET="very_real_client_secret_for_earthdaily_analytics_platform"
AUTH_TOKEN_URL="very_real_url_for_connecting_to_earthdaily_platform"
API_URL="very_real_api_url_for_reaching_earthdaily_dataset"
```

These values should be provided to you, but if you do not have them, please ask someone in the know.

(2) jazz hands?

Now that we have that squared away we are going to start by installing some required Python packages for this notebook.

In [ ]:
%pip install folium fsspec geopandas mapclassify python-dotenv pystac-client shapely

In [ ]:
from datetime import datetime
from pathlib import Path
import json
import os
import requests

from dotenv import load_dotenv
from pystac_client import Client
from pystac.item import Item
from shapely.geometry import shape

import cv2
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import tifffile as tiff

## Areas of Interest

To start, we need to create an area of interest that we want to analyze further.

If you have an area in mind already that you want to explore and learn more about, you can create your own GeoJSON file at [geojson.io](https://geojson.io/) or using an open source service like [QGIS](https://qgis.org/en/site/).

You can also download pre-built Shapefiles from services like [OpenStreetMap](https://download.geofabrik.de/) or from some of the resources mentioned in [this post by Carleton University](https://library.carleton.ca/find/gis/geospatial-data/shapefiles-canada-united-states-and-world).

We are going to start with a GeoJSON polygon that I've pre-built using geojson.io's interface that covers the some of Vancouver, BC, Canada and will inspect some Sentinel-2 tiles from the EarthDaily platform.

In [ ]:
# Example GeoJSON geometry
geojson_geometry = {
    # I drew a Polygon at geojson.io and put the contents of the `geometry` attribute here
    "type": "Polygon",
    "coordinates": [
        [
            [-123.19443904574592, 49.24932487550103],
            [-123.17802762978633, 49.13635374192958],
            [-123.03251301430745, 49.04966338761224],
            [-122.73054282797881, 49.06113525720673],
            [-122.68349674929038, 49.13420622071877],
            [-122.77868300152072, 49.255752174748864],
            [-122.98984330819309, 49.29073056863652],
            [-123.19443904574592, 49.24932487550103],
        ]
    ],
}

# Convert the GeoJSON geometry into a shapely geometry
shapely_geometry = shape(geojson_geometry)

# Please Note: You can also create, download, or import your own Shapefile to use with GeoPandas
# and create a GeoDataFrame from it. To do this see the docs at:
# https://geopandas.org/en/stable/docs/user_guide/io.html#reading-spatial-data
# and take a look at the geopandas read_file method.

# Create a GeoDataFrame with the shapely geometry
# Note that the Coordinate Reference System (CRS) defines how the two-dimensional,
# projected geometry relates to real world locations.
gdf = gpd.GeoDataFrame([{"geometry": shapely_geometry}], crs="EPSG:4326")

# Calculate the centroid of your GeoDataFrame to center the map
centroid = gdf.geometry.centroid.unary_union.centroid

# Create a folium map centered on the centroid of your shapefile
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=10)

# Add the GeoDataFrame as a layer to the folium map
folium.GeoJson(gdf, name="geojson").add_to(m)

# Add layer control to toggle the geojson layer
folium.LayerControl().add_to(m)

# Display the map
m

## Downloading Data

To download Sentinel-2 data from the Earth Platform we are going to use the [SpatioTemporal Asset Catalog (STAC)](https://stacspec.org/en) interface - a common language used to describe geospatial information. To do this, we need to authenticate with this platform using our precious credentials and then we can use the GeoDataFrame object we've already created to download data from the platform.

We demonstrate how to authorization token from this service in the following cell.

In [ ]:
load_dotenv(
    # You will need to create this file with your
    # CLIENT_ID, CLIENT_SECRET, AUTH_TOKEN_URL, and API_URL
    dotenv_path="secrets.env"
)

CLIENT_ID = os.environ.get("CLIENT_ID")
CLIENT_SECRET = os.environ.get("CLIENT_SECRET")
AUTH_TOKEN_URL = os.environ.get("AUTH_TOKEN_URL")
API_URL = os.environ.get("API_URL")


def get_new_token(client_id: str, client_secret: str, auth_token_url: str):
    """
    Authenticate with the Earth Platform and obtain a new access token.
    """
    token_req_payload = {"grant_type": "client_credentials"}
    token_response = requests.post(
        auth_token_url,
        data=token_req_payload,
        allow_redirects=False,
        auth=(client_id, client_secret),
    )
    token_response.raise_for_status()  # Raise an exception if the request failed

    tokens = json.loads(token_response.text)
    return tokens["access_token"]


token = get_new_token(CLIENT_ID, CLIENT_SECRET, AUTH_TOKEN_URL)
# Open a client to the STAC API
client = Client.open(API_URL, headers={"Authorization": f"Bearer {token}"})

## Query the STAC API

Now that we have established a connection to the client, we can use the geometry we've selected and obtain Sentinel-2 tiles for that area of interest. We are going to start by returning the available items for our area of interest within our query constraints and will then decide which tiles we want to download for further use.

In [ ]:
def get_sentinel2_data(
    client: Client,
    aoi: dict,
    start_date: str,
    end_date: str,
    cloud_cover: float = 1,
    max_items: int = 500,
):
    """
    Download Sentinel-2 data from the Earth Platform.
    """
    query = client.search(
        collections=["sentinel-2-l2a"],
        datetime=f"{start_date}T00:00:00.000000Z/{end_date}T00:00:00.000000Z",  # 2023-07-10T00:00:00.000000Z/2023-07-20T00:00:00.000000Z
        intersects=aoi,  # The area of interest; you can also query by bbox, or other geometry
        query={"eo:cloud_cover": {"lte": cloud_cover}},
        sortby=[
            {
                "field": "properties.eo:cloud_cover",
                "direction": "asc",
            },  # Sort by cloud cover from lowest to highest
        ],
        limit=max_items,  # This is the number of items to be returned per page
        max_items=max_items,  # This is number of items to page over
    )

    items = list(query.items())
    if len(items) == 0:
        raise Exception(
            "No items found, try enlarging search area or increasing cloud cover threshold."
        )
    print(f"Found: {len(items):d} tiles.")

    # Convert STAC items into a GeoJSON FeatureCollection
    stac_json = query.item_collection_as_dict()
    gdf = gpd.GeoDataFrame.from_features(stac_json, crs="EPSG:4326")

    return items, gdf


start_date = "2023-06-01"  # June 1, 2023
end_date = "2023-08-15"  # August 15, 2023

sentinel_items, sentinel_gdf = get_sentinel2_data(
    client,
    shapely_geometry,
    start_date,
    end_date,
    cloud_cover=10,
    max_items=50,
)
print(f"Found {len(sentinel_items)} Sentinel-2 items/tiles.")

In [ ]:
# Take a look at the tiles found for the given query on the map
sentinel_gdf.explore(color="green")

In [ ]:
# The items object contains a list of STAC items which includes
# metadata about the satellite imagery and links to the actual data.
# You can access the first item in the list like this:
sentinel_items[0]

## Download our tiles

Now we want to download our Sentinel-2 tiles from EarthDaily's Earth Store. In this example we are going to download all of the high, medium, and low resolution bands that are captured by the Sentinel-2 satellites. To learn a bit more about the significance of each captured band we are downloading, see this tutorial [here](https://gisgeography.com/sentinel-2-bands-combinations/).

Note that the information we download in the below cells does not exhaust the available data for each tile. You can find links to different files like class labels and metadata on the tile by downloading more files found for a given Sentinel-2 item.

In [ ]:
# This cell creates utility functions to download the files associated with a STAC item to a local
# file system.

def download_file(href: str, outpath: Path):
    """
    Given a URL, download the file to the specified path.
    """
    with requests.get(href, stream=True) as r:
        r.raise_for_status()
        with open(outpath, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)


def download_files_for_item(
    item: Item, asset_dict: dict[str, str], outpath: Path, debug: bool = True
) -> bool:
    """
    Save all files of interest for a given item.

    If one file fails to download return False, otherwise return True.
    """
    if not outpath.exists():
        outpath.mkdir(exist_ok=True, parents=True)

    for key, value in asset_dict.items():
        if debug:
          print(f"Downloading {key} and relabeling to {value}")
        if key in item.assets:
            if key in ["tileinfo_metadata"]:
                file_outpath = outpath / f"{value}.json"
            else:
                file_outpath = outpath / f"{value}.tiff"
            if not file_outpath.exists():
                try:
                    download_file(item.assets[key].href, file_outpath)
                except requests.ConnectionError:
                    print(
                        f"Failed to download {item.assets[key].href} for item {item.id}"
                    )
                    return False
                except requests.exceptions.ReadTimeout:
                    print(
                        f"Experienced a read timeout for {item.assets[key].href} for item {item.id}"
                    )
                    return False
            else:
                print(f"Skipping {item.assets[key].href} as it already exists.")

    return True

In [ ]:
high_resolution_bands = {"red": "B04", "green": "B03", "blue": "B02", "nir": "B08"}
mid_resolution_bands = {
    "rededge1": "B05",
    "rededge2": "B06",
    "rededge3": "B07",
    "nir08": "B8A",
    "swir16": "B11",
    "swir22": "B12",
}
low_resolution_bands = {"coastal": "B01", "nir09": "B09"}
other_files = {
    "scl": "scl",  # Scene Classification Map
    "tileinfo_metadata": "metadata",  # Tile Metadata
}
all_download_files = { # Modify this variable to change the files that are downloaded
    **high_resolution_bands,
    **mid_resolution_bands,
    **low_resolution_bands,
    **other_files,
}


def download_and_tile_files(items: list[Item], download_files: dict[str, str], output_dir: Path):
    """
    Given a GeoDataFrame of items and a list of STAC items, download the
    files to a given output directory.

    Parameters:
      items: A list of items that correspond to tiles found in the GeoDataFrame and include paths to files to be
        downloaded in this function.
      download_files: A dictionary of strings where the keys correspond to names of items on the Earth Platform
        and the values correspond to their Sentinel-2 name.
    """
    downloaded = 0
    for index, tile in enumerate(items):
        dt_obj = datetime.strptime(tile.properties["datetime"], "%Y-%m-%dT%H:%M:%S.%fZ")
        formatted_date = dt_obj.strftime("%Y%m%d")
        out_path = output_dir / tile.id / formatted_date
        downloaded = download_files_for_item(tile, download_files, out_path)

        if downloaded:
            downloaded += 1
        else:
            print(f"Unable to download file for item with id: {tile.id} at index: {index} in items list.")

    print(
        f"Downloaded all bands for {downloaded} tiles. Failed to download at least one "
        + f"band or file for {len(items) - downloaded} tiles."
    )


output_dir = Path("/content/sentinel_tiles") # Used for Google CoLab

# We are only going to download 2 tiles here, but feel free to modify this function
# call to download more data!
download_and_tile_files(sentinel_items[0:2], all_download_files, output_dir)

## Data Analysis

Now that we have downloaded atmospherically corrected Sentinel-2 data, we are going to display some common representations of the images that with any luck will serve as inspiration for doing something fancy.

In [ ]:
def display_images(image_dict):
    fig, axes = plt.subplots(2, 3, figsize=(12, 8))
    fig.tight_layout()

    for i, (title, image) in enumerate(image_dict.items()):
        row = i // 3
        col = i % 3
        axes[row, col].imshow(image)
        axes[row, col].set_title(title)
        axes[row, col].axis("off")

    plt.show()


def crop_center_square(arr, max_dim: int = 1000):
    """
    Selects up to the center max_pixels from an nxnxc input array, or n pixels if 1000 > n,
    where c is the number of channels (e.g., 3 for an RGB image).

    Parameters:
        arr (np.ndarray): The input array from which to select pixels, with shape (n, n, c).
        max_pixels (int): The maximum number of pixels to select if possible (default is 1000).

    Returns:
        np.ndarray: The selected portion of the input array, maintaining the channel dimension.
    """
    if arr.ndim > 3 or arr.ndim < 2:
        raise ValueError(
            "Unable to crop center of array with number of dimensions:"
            + f" {arr.ndim}, must have only 2 or 3 dimensions."
        )
    if max_dim > arr.shape[0]:
        return arr

    # Calculate starting indices to select the square from the center
    start_index = (arr.shape[0] - max_dim) // 2

    # Select the square, preserving the channel dimension
    if arr.ndim == 3:
        selected_square = arr[start_index:start_index + max_dim, start_index:start_index + max_dim, :]
    elif arr.ndim == 2:
        selected_square = arr[start_index:start_index + max_dim, start_index:start_index + max_dim]


    return selected_square


def brighten(band: np.ndarray):
    """Adjust image brightness using an alpha value (0 to 1). Higher alpha increases brightness, lower decreases."""
    
    alpha=0.2 # you can change this value 
    beta=0
    return np.clip(alpha*band+beta, 0,255)

def normalize(band: np.ndarray) -> np.ndarray:
    """
    First brighten the bands and then return normalized image 
    """
    band = brighten(band) 
    band_min, band_max = (band.min(), band.max())

    return ((band-band_min)/((band_max - band_min)))


def display_transformed_images(data_path: Path, num_pixels: int = 1500):
    """
    Generate Sentinel-2 images from the given filepath. The returned images are as follows:
       NDVI: Normalized Difference Vegetation Index
       NDBI: Normalized Difference Built-up Index
       NDWI: Normalized Difference Water Index
       False Color: B08, B04, B03
       Mask: The class labels for each pixel

    Please Note: If the size of the input

    Note: If the input tile contains 0 values (ie. black squares in the tiles),
      a warning will be printed due to division by zero.
    """
    # Unpack the bands
    B02 = crop_center_square(
        normalize(
            tiff.imread(f"{data_path}/B02.tiff"),),
        num_pixels
    )
    B03 = crop_center_square(
        normalize(
            tiff.imread(f"{data_path}/B03.tiff"),),
        num_pixels
    )
    B04 = crop_center_square(
        normalize(
            tiff.imread(f"{data_path}/B04.tiff"),),
        num_pixels
    )
    B08 = crop_center_square(
        normalize(
            tiff.imread(f"{data_path}/B08.tiff"),),
        num_pixels
    )
    B11 = crop_center_square(
        normalize(
            cv2.resize(
                tiff.imread(f"{data_path}/B11.tiff"),
                None,
                fx=2,
                fy=2,
                interpolation=cv2.INTER_CUBIC,
            ),),
        num_pixels
    )
    B12 = crop_center_square(
        normalize(
            cv2.resize(
                tiff.imread(f"{data_path}/B12.tiff"),
                None,
                fx=2,
                fy=2,
                interpolation=cv2.INTER_CUBIC,
            ),),
        num_pixels
    )

    # Calculate NDVI (Normalized Difference Vegetation Index)
    NDVI = (B08 - B04) / (B08 + B04)

    # # Calculate NDBI (Normalized Difference Built-Up Index)
    NDBI = (B11 - B08) / (B11 + B08)

    # Calculate NDWI (Normalized Difference Water Index)
    NDWI = (B08 - B12) / (B08 + B12)

    # Calculate NBR (Normalized Burn Ratio)
    NBR = (B08 - B12) / (B08 + B12)

    # Create a color image using RGB bands
    RGB = np.stack([B04, B03, B02], axis=-1)

    # Create a false-color composite image
    false_color = np.stack([B08, B04, B03], axis=-1)
    images = {
        "NDVI": NDVI,
        "NDBI": NDBI,
        "NDWI": NDWI,
        "NBR": NBR,
        "RGB": RGB,
        "False Color": false_color,
    }

    display_images(images)

    return images

# Note: Please update data_path based on the root directory of one tile
data_path = Path(
    "/content/sentinel_tiles/S2B_10UEV_20230719_0_L2A/20230719"
)
display_pixels = 2000 # Number of center pixels to display for image

images = display_transformed_images(data_path, num_pixels=display_pixels)

## Display Scene Classification Map

While applying atmospheric corrections to Sentinel-2 data the Level-2A Algorithm also generates a map that classifies pixels based on the captured data for that pixel. More information on this algorithm can be found [here](https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-2-msi/level-2a/algorithm-overview).

In the below cell we display the scene classification map for the obtain image and the corresponding class names.

In [ ]:
import matplotlib.patches as mpatches

SCENE_CLASSES = { # Source: https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/scene-classification/
    0: [0, 0, 0],        # No Data (Missing data) - black
    1: [255, 0, 0],      # Saturated or defective pixel - red
    2: [47, 47, 47],     # Topographic casted shadows - very dark grey
    3: [100, 50, 0],     # Cloud shadows - dark brown
    4: [0, 160, 0],      # Vegetation - green
    5: [255, 230, 90],   # Not-vegetated - dark yellow
    6: [0, 0, 255],      # Water (dark and bright) - blue
    7: [128, 128, 128],  # Unclassified - dark grey
    8: [192, 192, 192],  # Cloud medium probability - grey
    9: [255, 255, 255],  # Cloud high probability - white
    10: [100, 200, 255], # Thin cirrus - very bright blue
    11: [255, 150, 255], # Snow or ice - very bright pink
}
LABEL_NAMES = {
    0: "No Data (Missing data)",
    1: "Saturated or defective pixel",
    2: "Topographic casted shadows",
    3: "Cloud shadows",
    4: "Vegetation",
    5: "Not-vegetated",
    6: "Water (dark and bright)",
    7: "Unclassified",
    8: "Cloud medium probability",
    9: "Cloud high probability",
    10: "Thin cirrus",
    11: "Snow or ice",
}


def get_scene_classification_map(data_path: Path, class_map: dict[int, list[int]] = SCENE_CLASSES):
    """
    Read in the Scene Classification Map downloaded from EarthDaily and return
    colorized array.
    """
    scl = tiff.imread(data_path)
    color_image = np.array([SCENE_CLASSES[value] for row in scl for value in row])
    color_image = color_image.reshape(*scl.shape, 3)

    return color_image


def display_scl_map(scl: np.ndarray):
    """
    Given the scene classification map and the predefined label names and legend
    information, display the Scene Classification Map and the corresponding class
    labels.
    """
    patches = [mpatches.Patch(color=np.array(color)/255.0, label=LABEL_NAMES[i]) for i, color in SCENE_CLASSES.items()]

    # Display the color image with legend
    plt.figure(figsize=(12, 4))
    plt.imshow(scl)
    plt.axis('off')  # Hide the axis to emphasize the image
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.tight_layout()
    plt.show()


scl = get_scene_classification_map(f"{data_path}/scl.tiff")
display_scl_map(scl)